In [1]:
%%writefile magnitude_comparison.cu
#include <iostream>
#include <cmath>
#include <chrono>  // Para medir o tempo de execução
#include <cuda.h>

// Kernel para calcular os quadrados dos elementos do vetor na GPU
__global__ void square_elements(float *v, float *result, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        result[idx] = v[idx] * v[idx];  // Elevar ao quadrado cada elemento
    }
}

// Função para calcular a magnitude na GPU
float magnitude_gpu(float *h_v, int N) {
    float *d_v, *d_result;

    // Alocar memória na GPU
    cudaMalloc(&d_v, N * sizeof(float));
    cudaMalloc(&d_result, N * sizeof(float));

    // Copiar vetor da CPU para a GPU
    cudaMemcpy(d_v, h_v, N * sizeof(float), cudaMemcpyHostToDevice);

    // Configuração dos blocos e threads
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Executar o kernel para calcular os quadrados dos elementos
    square_elements<<<blocksPerGrid, threadsPerBlock>>>(d_v, d_result, N);

    // Copiar o resultado de volta para a CPU
    float *h_result = new float[N];
    cudaMemcpy(h_result, d_result, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Somar todos os quadrados e calcular a magnitude
    float sum_of_squares = 0.0f;
    for (int i = 0; i < N; i++) {
        sum_of_squares += h_result[i];
    }

    // Liberar memória na GPU
    cudaFree(d_v);
    cudaFree(d_result);
    delete[] h_result;

    // Retornar a magnitude
    return std::sqrt(sum_of_squares);
}

// Função para calcular a magnitude na CPU
float magnitude_cpu(float *v, int N) {
    float sum_of_squares = 0.0f;
    for (int i = 0; i < N; i++) {
        sum_of_squares += v[i] * v[i];
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    // Tamanho do vetor
    int N = 1000000;

    // Alocar e inicializar o vetor com valores aleatórios
    float *v = new float[N];
    for (int i = 0; i < N; i++) {
        v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Calcular a magnitude na CPU e medir o tempo
    auto start_cpu = std::chrono::high_resolution_clock::now();
    float mag_cpu = magnitude_cpu(v, N);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_cpu = end_cpu - start_cpu;
    std::cout << "Magnitude na CPU: " << mag_cpu << std::endl;
    std::cout << "Tempo de execução na CPU: " << duration_cpu.count() << " segundos" << std::endl;

    // Calcular a magnitude na GPU e medir o tempo
    auto start_gpu = std::chrono::high_resolution_clock::now();
    float mag_gpu = magnitude_gpu(v, N);
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_gpu = end_gpu - start_gpu;
    std::cout << "Magnitude na GPU: " << mag_gpu << std::endl;
    std::cout << "Tempo de execução na GPU: " << duration_gpu.count() << " segundos" << std::endl;

    // Liberar memória da CPU
    delete[] v;

    return 0;
}


Writing magnitude_comparison.cu


In [3]:
!nvcc magnitude_comparison.cu -o magnitude_comparison


In [5]:
!./magnitude_comparison

Magnitude na CPU: 577.228
Tempo de execução na CPU: 0.00386753 segundos
Magnitude na GPU: 0
Tempo de execução na GPU: 0.0049066 segundos


In [6]:
%%writefile variance_comparison.cu
#include <iostream>
#include <cmath>
#include <chrono>  // Para medir o tempo de execução
#include <cuda.h>

// Kernel para calcular a média de um vetor
__global__ void mean_kernel(float *v, float *mean, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        atomicAdd(mean, v[idx] / N);  // Atomic para garantir a soma correta entre as threads
    }
}

// Kernel para calcular a diferença ao quadrado para a variância
__global__ void variance_kernel(float *v, float mean, float *result, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        float diff = v[idx] - mean;
        atomicAdd(result, diff * diff / N);  // Soma das diferenças ao quadrado
    }
}

// Função que calcula a variância em etapas separadas na GPU
float variance_separate_gpu(float *h_v, int N) {
    float *d_v, *d_mean, *d_variance;
    float h_mean = 0.0f, h_variance = 0.0f;

    // Alocar memória na GPU
    cudaMalloc(&d_v, N * sizeof(float));
    cudaMalloc(&d_mean, sizeof(float));
    cudaMalloc(&d_variance, sizeof(float));

    // Copiar o vetor da CPU para a GPU
    cudaMemcpy(d_v, h_v, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mean, &h_mean, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_variance, &h_variance, sizeof(float), cudaMemcpyHostToDevice);

    // Configurar grid e block
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Calcular a média
    mean_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_v, d_mean, N);
    cudaMemcpy(&h_mean, d_mean, sizeof(float), cudaMemcpyDeviceToHost);

    // Calcular a variância
    variance_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_v, h_mean, d_variance, N);
    cudaMemcpy(&h_variance, d_variance, sizeof(float), cudaMemcpyDeviceToHost);

    // Liberar a memória da GPU
    cudaFree(d_v);
    cudaFree(d_mean);
    cudaFree(d_variance);

    return h_variance;
}

// Fusion Kernel: Calcular variância em uma única etapa
__global__ void variance_fusion_kernel(float *v, float *result, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        // Primeira fase: cálculo da média
        __shared__ float mean;
        mean = 0.0f;
        atomicAdd(&mean, v[idx] / N);
        __syncthreads();  // Espera a finalização do cálculo da média

        // Segunda fase: cálculo da variância em uma única operação
        float diff = v[idx] - mean;
        atomicAdd(result, diff * diff / N);
    }
}

// Função que calcula a variância usando fusion kernel na GPU
float variance_fusion_gpu(float *h_v, int N) {
    float *d_v, *d_variance;
    float h_variance = 0.0f;

    // Alocar memória na GPU
    cudaMalloc(&d_v, N * sizeof(float));
    cudaMalloc(&d_variance, sizeof(float));

    // Copiar o vetor da CPU para a GPU
    cudaMemcpy(d_v, h_v, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_variance, &h_variance, sizeof(float), cudaMemcpyHostToDevice);

    // Configurar grid e block
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Executar o kernel de variância com fusão
    variance_fusion_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_v, d_variance, N);
    cudaMemcpy(&h_variance, d_variance, sizeof(float), cudaMemcpyDeviceToHost);

    // Liberar a memória da GPU
    cudaFree(d_v);
    cudaFree(d_variance);

    return h_variance;
}

// Função para calcular a variância na CPU
float variance_cpu(float *v, int N) {
    float mean = 0.0f;
    for (int i = 0; i < N; i++) {
        mean += v[i];
    }
    mean /= N;

    float variance = 0.0f;
    for (int i = 0; i < N; i++) {
        float diff = v[i] - mean;
        variance += diff * diff;
    }
    return variance / N;
}

int main() {
    // Tamanhos dos vetores
    int N1 = 100000;
    int N2 = 1000000;

    // Vetores na CPU
    float *v1 = new float[N1];
    float *v2 = new float[N2];

    // Inicializar os vetores com valores aleatórios
    for (int i = 0; i < N1; i++) {
        v1[i] = static_cast<float>(rand()) / RAND_MAX;
    }
    for (int i = 0; i < N2; i++) {
        v2[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Variância na CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();
    float var_cpu1 = variance_cpu(v1, N1);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_cpu1 = end_cpu - start_cpu;

    // Variância separada na GPU
    auto start_gpu_sep = std::chrono::high_resolution_clock::now();
    float var_gpu_sep1 = variance_separate_gpu(v1, N1);
    auto end_gpu_sep = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_gpu_sep1 = end_gpu_sep - start_gpu_sep;

    // Variância com fusion kernel na GPU
    auto start_gpu_fusion = std::chrono::high_resolution_clock::now();
    float var_gpu_fusion1 = variance_fusion_gpu(v1, N1);
    auto end_gpu_fusion = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_gpu_fusion1 = end_gpu_fusion - start_gpu_fusion;

    // Imprimir resultados
    std::cout << "Tamanho N1 = " << N1 << std::endl;
    std::cout << "Variância CPU: " << var_cpu1 << " | Tempo CPU: " << duration_cpu1.count() << " segundos" << std::endl;
    std::cout << "Variância GPU (etapas separadas): " << var_gpu_sep1 << " | Tempo GPU: " << duration_gpu_sep1.count() << " segundos" << std::endl;
    std::cout << "Variância GPU (fusion kernel): " << var_gpu_fusion1 << " | Tempo GPU: " << duration_gpu_fusion1.count() << " segundos" << std::endl;

    // Testar com o vetor maior N2
    // Variância CPU
    start_cpu = std::chrono::high_resolution_clock::now();
    float var_cpu2 = variance_cpu(v2, N2);
    end_cpu = std::chrono::high_resolution_clock::now();
    duration_cpu1 = end_cpu - start_cpu;

    // Variância separada na GPU
    start_gpu_sep = std::chrono::high_resolution_clock::now();
    float var_gpu_sep2 = variance_separate_gpu(v2, N2);
    end_gpu_sep = std::chrono::high_resolution_clock::now();
    duration_gpu_sep1 = end_gpu_sep - start_gpu_sep;

    // Variância com fusion kernel na GPU
    start_gpu_fusion = std::chrono::high_resolution_clock::now();
    float var_gpu_fusion2 = variance_fusion_gpu(v2, N2);
    end_gpu_fusion = std::chrono::high_resolution_clock::now();
    duration_gpu_fusion1 = end_gpu_fusion - start_gpu_fusion;

    // Imprimir resultados
    std::cout << "\nTamanho N2 = " << N2 << std::endl;
    std::cout << "Variância CPU: " << var_cpu2 << " | Tempo CPU: " << duration_cpu1.count() << " segundos" << std::endl;
    std::cout << "Variância GPU (etapas separadas): " << var_gpu_sep2 << " | Tempo GPU: " << duration_gpu_sep1.count() << " segundos" << std::endl;
    std::cout << "Variância GPU (fusion kernel): " << var_gpu_fusion2 << " | Tempo GPU: " << duration_gpu_fusion1.count() << " segundos" << std::endl;

    // Liberar a memória
    delete[] v1;
    delete[] v2;

    return 0;
}


Writing variance_comparison.cu


In [7]:
!nvcc variance_comparison.cu -o variance_comparison


In [8]:
!./variance_comparison


Tamanho N1 = 100000
Variância CPU: 0.083289 | Tempo CPU: 0.000694844 segundos
Variância GPU (etapas separadas): 0 | Tempo GPU: 0.000238345 segundos
Variância GPU (fusion kernel): 0 | Tempo GPU: 4.5e-07 segundos

Tamanho N2 = 1000000
Variância CPU: 0.0832533 | Tempo CPU: 0.00689089 segundos
Variância GPU (etapas separadas): 0 | Tempo GPU: 5.149e-06 segundos
Variância GPU (fusion kernel): 0 | Tempo GPU: 3.32e-07 segundos


In [9]:
%%writefile variance_comparison2.cu
#include <iostream>
#include <cmath>
#include <chrono>
#include <cuda.h>

// Kernel para calcular a média de um vetor
__global__ void mean_kernel(float *v, float *mean, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        atomicAdd(mean, v[idx] / N);
    }
}

// Kernel para calcular a diferença ao quadrado para a variância
__global__ void variance_kernel(float *v, float mean, float *result, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        float diff = v[idx] - mean;
        atomicAdd(result, diff * diff / N);
    }
}

// Função que calcula a variância em etapas separadas na GPU
float variance_separate_gpu(float *h_v, int N) {
    float *d_v, *d_mean, *d_variance;
    float h_mean = 0.0f, h_variance = 0.0f;

    cudaMalloc(&d_v, N * sizeof(float));
    cudaMalloc(&d_mean, sizeof(float));
    cudaMalloc(&d_variance, sizeof(float));

    cudaMemcpy(d_v, h_v, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mean, &h_mean, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_variance, &h_variance, sizeof(float), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    mean_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_v, d_mean, N);
    cudaMemcpy(&h_mean, d_mean, sizeof(float), cudaMemcpyDeviceToHost);

    variance_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_v, h_mean, d_variance, N);
    cudaMemcpy(&h_variance, d_variance, sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_v);
    cudaFree(d_mean);
    cudaFree(d_variance);

    return h_variance;
}

// Fusion Kernel
__global__ void variance_fusion_kernel(float *v, float *result, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        __shared__ float mean;
        mean = 0.0f;
        atomicAdd(&mean, v[idx] / N);
        __syncthreads();

        float diff = v[idx] - mean;
        atomicAdd(result, diff * diff / N);
    }
}

// Função que calcula a variância usando fusion kernel na GPU
float variance_fusion_gpu(float *h_v, int N) {
    float *d_v, *d_variance;
    float h_variance = 0.0f;

    cudaMalloc(&d_v, N * sizeof(float));
    cudaMalloc(&d_variance, sizeof(float));

    cudaMemcpy(d_v, h_v, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_variance, &h_variance, sizeof(float), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    variance_fusion_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_v, d_variance, N);
    cudaMemcpy(&h_variance, d_variance, sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_v);
    cudaFree(d_variance);

    return h_variance;
}

// Função para calcular a variância na CPU
float variance_cpu(float *v, int N) {
    float mean = 0.0f;
    for (int i = 0; i < N; i++) {
        mean += v[i];
    }
    mean /= N;

    float variance = 0.0f;
    for (int i = 0; i < N; i++) {
        float diff = v[i] - mean;
        variance += diff * diff;
    }
    return variance / N;
}

int main() {
    // Tamanhos dos vetores
    int sizes[] = {100000, 1000000, 10000000, 100000000};
    int num_sizes = sizeof(sizes) / sizeof(sizes[0]);

    // Iterar sobre cada tamanho de vetor
    for (int i = 0; i < num_sizes; i++) {
        int N = sizes[i];

        // Vetor na CPU
        float *v = new float[N];

        // Inicializar o vetor com valores aleatórios
        for (int i = 0; i < N; i++) {
            v[i] = static_cast<float>(rand()) / RAND_MAX;
        }

        std::cout << "\nTamanho N = " << N << std::endl;

        // Variância na CPU
        auto start_cpu = std::chrono::high_resolution_clock::now();
        float var_cpu = variance_cpu(v, N);
        auto end_cpu = std::chrono::high_resolution_clock::now();
        std::chrono::duration<float> duration_cpu = end_cpu - start_cpu;
        std::cout << "Variância CPU: " << var_cpu << " | Tempo CPU: " << duration_cpu.count() << " segundos" << std::endl;

        // Variância separada na GPU
        auto start_gpu_sep = std::chrono::high_resolution_clock::now();
        float var_gpu_sep = variance_separate_gpu(v, N);
        auto end_gpu_sep = std::chrono::high_resolution_clock::now();
        std::chrono::duration<float> duration_gpu_sep = end_gpu_sep - start_gpu_sep;
        std::cout << "Variância GPU (etapas separadas): " << var_gpu_sep << " | Tempo GPU: " << duration_gpu_sep.count() << " segundos" << std::endl;

        // Variância com fusion kernel na GPU
        auto start_gpu_fusion = std::chrono::high_resolution_clock::now();
        float var_gpu_fusion = variance_fusion_gpu(v, N);
        auto end_gpu_fusion = std::chrono::high_resolution_clock::now();
        std::chrono::duration<float> duration_gpu_fusion = end_gpu_fusion - start_gpu_fusion;
        std::cout << "Variância GPU (fusion kernel): " << var_gpu_fusion << " | Tempo GPU: " << duration_gpu_fusion.count() << " segundos" << std::endl;

        // Liberar memória
        delete[] v;
    }

    return 0;
}


Writing variance_comparison2.cu


In [10]:
!nvcc variance_comparison2.cu -o variance_comparison2

In [12]:
!./variance_comparison

Tamanho N1 = 100000
Variância CPU: 0.083289 | Tempo CPU: 0.000648772 segundos
Variância GPU (etapas separadas): 0 | Tempo GPU: 0.000262605 segundos
Variância GPU (fusion kernel): 0 | Tempo GPU: 5.31e-07 segundos

Tamanho N2 = 1000000
Variância CPU: 0.0832533 | Tempo CPU: 0.00675333 segundos
Variância GPU (etapas separadas): 0 | Tempo GPU: 4.545e-06 segundos
Variância GPU (fusion kernel): 0 | Tempo GPU: 5.2e-07 segundos
